In [1]:
%load_ext autoreload
%autoreload 2
from new_aeb_gplvm import *
from utils.data_generator import DataGenerator
from utils.myutils import Utils
from tqdm import trange
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import random
from tqdm import tqdm_notebook
random.seed(42)
import json

2023-11-28 02:36:16.381826: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-28 02:36:16.381856: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-28 02:36:16.382942: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-28 02:36:16.389365: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-28 02:36:17.219882: W tensorflow/comp

In [2]:
datagenerator = DataGenerator()
utils = Utils()

In [3]:
# dataset and model list / dict

In [14]:
def create_dataframe(dataset):
    datagenerator.dataset = dataset
    data = datagenerator.generator(
        la=1.00,
        realistic_synthetic_mode=None,
        stdscale=True,
        minmax=False,
        noise_type=None,
    )
    Y_train = torch.tensor(data["X_train"], dtype=torch.float32)
    Y_test = torch.tensor(data["X_test"], dtype=torch.float32)
    lb_train = torch.tensor(data["y_train"], dtype=torch.float32)
    lb_test = torch.tensor(data["y_test"], dtype=torch.float32)
    #idx_n = np.where(lb_train == 0)[0]
    #Y_train = Y_train[idx_n]
    #lb_train = lb_train[idx_n]
    return Y_train, Y_test, lb_train, lb_test

In [15]:
Y_train, Y_test, lb_train, lb_test = create_dataframe('01_ALOI')

subsampling for dataset 01_ALOI...
current noise type: None
{'Samples': 10000, 'Features': 27, 'Anomalies': 302, 'Anomalies Ratio(%)': 3.02}


In [27]:
with open("best_experiments.json") as file:
    experiments = json.load(file)

In [28]:
for experiment in tqdm_notebook(experiments):
    if "train_noise" not in experiment.keys():
        Y_train, Y_test, lb_train, lb_test = create_dataframe(experiment["dataset"])
        n_train = len(Y_train)
        data_dim = Y_train.shape[1]
        latent_dim = experiment["latent_dim"]
        nn_layers = tuple(map(int, experiment["layers"].split(",")))
        kernel = experiment["kernel"]
        
        n_inducing = 50
        n_epochs = 1000
        lr = 0.01
        batch_size = 128
        
        try:
            gplvm = AD_GPLVM(latent_dim, n_inducing, n_epochs, nn_layers, lr, batch_size)
            gplvm.fit(Y_train)
            score = gplvm.predict_score(Y_test)
            metrics = utils.metric(y_true=lb_test, y_score=score)
            
            experiment["auc_roc"] = metrics['aucroc']
            experiment["auc_pr"] = metrics['aucpr']
        except:
            experiment['auc_roc'] = 0.00
            experiment['auc_pr'] = 0.00
        
        experiment["lr"] = lr
        experiment["batch_size"] = batch_size
        experiment["n_inducing"] = n_inducing
        experiment["n_epochs"] =  n_epochs
        experiment["train_noise"] =  float(lb_train.sum()/len(Y_train))
        
        with open("experiment_results_noisy_gplvm.json", "w") as file:
            json.dump(experiments, file)

/tmp/ipykernel_32706/1028631601.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for experiment in tqdm_notebook(experiments):


  0%|          | 0/47 [00:00<?, ?it/s]

subsampling for dataset 01_ALOI...
current noise type: None
{'Samples': 10000, 'Features': 27, 'Anomalies': 302, 'Anomalies Ratio(%)': 3.02}
Kernel não escolhido!



Loss: 31.87, iter no: 500: 100%|██████████| 1000/1000 [00:56<00:00, 17.68it/s]
                                                                              /home/ec2-user/SageMaker/OE-GPLVM/new_aeb_gplvm.py:302: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1686274778240/work/aten/src/ATen/native/TensorShape.cpp:3571.)
  klu_expanded = ll_shape.T.add_(klu).sum(-1).T.div((self.n_train))


current noise type: None
{'Samples': 7200, 'Features': 6, 'Anomalies': 534, 'Anomalies Ratio(%)': 7.42}
Kernel não escolhido!


...


Loss: 16.77, iter no: 500: 100%|██████████| 1000/1000 [00:13<00:00, 75.25it/s]
                                                                              

generating duplicate samples for dataset 46_WPBC...
current noise type: None
{'Samples': 1000, 'Features': 33, 'Anomalies': 249, 'Anomalies Ratio(%)': 24.9}
Kernel não escolhido!



Loss: 48.86, iter no: 500: 100%|██████████| 1000/1000 [00:18<00:00, 55.36it/s]
                                                                              

current noise type: None
{'Samples': 1484, 'Features': 8, 'Anomalies': 507, 'Anomalies Ratio(%)': 34.16}
Kernel não escolhido!



Loss: 12.15, iter no: 500: 100%|██████████| 1000/1000 [00:13<00:00, 76.14it/s]
                                                                              